# STAC Challenge automatized
Test on Accra 1: 
https://oin-hotosm.s3.amazonaws.com/5b694a0f4b87366cc0f0fa70/0/5b694a0f4b8736ebfff0fa71.tif <br><br>

All of this is made assuming that the structure of the folder would be: <br><br>

stac_challenge_folder / accra_1 / accra_1_drains.geojson <br><br> 
stac_challenge_folder / accra_1 / accra_1_roads.geojson <br><br> 
stac_challenge_folder / accra_1 / accra_1_buildings.geojson <br><br> 
stac_challenge_folder / accra_2 / accra_2_drains.geojson <br><br> 
...<br><br>

And that the links to the tiffiles would be listed in tiffiles (see below)

In [4]:
from satstac import Catalog
from satstac import Collection
from satstac import Item
import rasterio
import shapely
import sys,os,os.path
import json

In [5]:
#Where to store the collection and the items:
stac_challenge_folder = '/Users/axel/Desktop/STAC_Challenge_2'
catalog_address = stac_challenge_folder + '/' + 'catalog.json'

#collection_address = stac_challenge_folder + '/' + 'collection.json'

tiffiles = {"accra_1": "https://oin-hotosm.s3.amazonaws.com/5b694a0f4b87366cc0f0fa70/0/5b694a0f4b8736ebfff0fa71.tif"}

### Info for the root catalog (hence generic info about the challenge):

In [6]:
stac_version = "??"
catalog_id = 'challenge_catalog'
catalog_title = 'Challlenge OpenML'
catalog_description = 'Data for the ML challenge, in the STAC format'
catalog_links = [ {
      "rel": "self",
      "href": catalog_address
    },
        {
      "rel": "root",
      "href": catalog_address
    },

]


### Generic Info for the collections (Info that will be in common for all of them)

In [7]:
license = "open" #open, proprietary?
version = "1.0"
providers = [
    {
      "name": "WB/OCA",
      "roles": [
        "producer",
        "licensor"
      ],
      "url": "url to OCA"
    }
  ]
basic_keywords = ["challenge", "world bank"]
extent = {
    "spatial": [
      -180.0,
      -56.0,
      180.0,
      83.0
    ],
    "temporal": [
      "2015-06-23T00:00:00Z",
        None
    ]
  }

id_collection_basic = "challenge_collection_for_"

basic_description = "The data available for the challenge formated in a STAC collection for "
basic_title = "Data for the challenge "

### Generic Info for the items (Info that will be in common for all of them)

In [8]:
type_item = "Feature"
properties_item = {
    "collection": "",
    "datetime": "2019-02-26T00:00:00Z",
    "td:title": "accra_1",
    "td:description": "test",
    "td:label_type": "test",
    "td:classes": [
      "building", "drains", "roads"
    ]
  }

## Step 1: Creating the Catalog


In [9]:
catalog_json = {
    "stac_version": stac_version,
    "id": catalog_id,
    "title": catalog_title,
    "description": catalog_description,
    "links": catalog_links
    
    
}
catalog = Catalog(catalog_json)
print(catalog.id)
# save as a root catalog
catalog.save_as(catalog_address)
print(catalog.id)
print(catalog.filename)
print(catalog.path)

challenge_catalog
challenge_catalog
/Users/axel/Desktop/STAC_Challenge_2/catalog.json
/Users/axel/Desktop/STAC_Challenge_2


## Functions to create the collections



In [25]:
def get_tags_collection(foldername):
    key = basic_keywords.copy()
    key.append(foldername)
    #collection_address = stac_challenge_folder +'/' + foldername + '_collection.json'
    collection_adress = stac_challenge_folder +'/' +id_collection_basic+folder+'/'+ folder + '_collection.json'
    links =[ {
          "rel": "self",
          "href": collection_address
        },
            {
          "rel": "root",
          "href": catalog_address
        }]
    title = basic_title + foldername
    description = basic_description + foldername
    id_collection = id_collection_basic + foldername
    return stac_version, id_collection, title, description, key, version, license, providers, extent, links

In [26]:
def create_corresponding_collection(stac_version, id_collection, title, description, keywords, version, license, providers, extent, links):
    # create a Colllection object with JSON
    collection_json = {
    "stac_version": "??",
    "id": id_collection,
    "title": title ,
    "description": description,
    "collection version": version,
    "keywords":keywords,
    "license": license,
    "version": version,
    "providers":providers,
    "extent":extent,
    "links": links
    }
    col = Collection(collection_json)
    print(col.id)
    return col

### test

In [27]:
print(get_tags_collection("accra_1"))

('??', 'challenge_collection_for_accra_1', 'Data for the challenge accra_1', 'The data available for the challenge formated in a STAC collection for accra_1', ['challenge', 'world bank', 'accra_1'], '1.0', 'open', [{'name': 'WB/OCA', 'roles': ['producer', 'licensor'], 'url': 'url to OCA'}], {'spatial': [-180.0, -56.0, 180.0, 83.0], 'temporal': ['2015-06-23T00:00:00Z', None]}, [{'rel': 'self', 'href': '/Users/axel/Desktop/STAC_Challenge_2/accra_1_collection.json'}, {'rel': 'root', 'href': '/Users/axel/Desktop/STAC_Challenge_2/catalog.json'}])


In [28]:
stac_version, id_collection, title, description, keywords, version, license, providers, extent, links = get_tags_collection("accra_1")
col = create_corresponding_collection(stac_version, id_collection, title, description, keywords, version, license, providers, extent, links)

print('Title:', col.title)
print('Collection Version:', col.version)
print('Keywords: ', col.keywords)
print('License:', col.license)
print('Providers:', col.providers)
print('Extent', col.extent)



collection_address = stac_challenge_folder +'/' + "accra_1" + '_collection.json'
col.save_as(collection_address)
print(col.id)
print(col.filename)
print(col.path)
collection = Collection.open(collection_address)

challenge_collection_for_accra_1
Title: Data for the challenge accra_1
Collection Version: 1.0
Keywords:  ['challenge', 'world bank', 'accra_1']
License: open
Providers: [{'name': 'WB/OCA', 'roles': ['producer', 'licensor'], 'url': 'url to OCA'}]
Extent {'spatial': [-180.0, -56.0, 180.0, 83.0], 'temporal': ['2015-06-23T00:00:00Z', None]}
challenge_collection_for_accra_1
/Users/axel/Desktop/STAC_Challenge_2/accra_1_collection.json
/Users/axel/Desktop/STAC_Challenge_2


## Functions to create the items

In [29]:
def get_bbox(directory, filename):
    with open(directory + '/'+ filename, 'r') as f:
        data = json.load(f)
        c1 = [];
        c2 = [];
        for f in data["features"]:
            c_partial = f["geometry"]["coordinates"][0]
            for x in c_partial:
                c1.append(x[0]) # first coordinate
                c2.append(x[1])
        bbox = [[min(c1),min(c2)],[min(c1),max(c2)],[max(c1),max(c2)],[max(c1),min(c2)],[min(c1),min(c2)]]
    return bbox

In [30]:
def get_tags_items(collection, foldername, filename, bbox):
    filename_split_1 = filename.split('.');
    filename_split = filename_split_1[0]
    id_item = filename_split
    links_item = [    {
      "rel": "self",
      "href": stac_challenge_folder+"/"+filename_split+"-item.json"
    },
    {
      "rel": "root",
      "href": collection.filename
    },]
    
    assets_item = {
    "raster": {
        "title": "image",
        "href": tiffiles[filename.split('_')[0]+'_'+filename.split('_')[1]],
        "type": "image/vnd.stac.geotiff; cloud-optimized=true"
    },
    "vector": {
      "title": filename_split,
      "href": stac_challenge_folder+'/'+foldername +'/'+filename,
      "type": "application/geo+json"
    }
    }
    key = basic_keywords.copy()
    key.append(filename_split[0]+'_'+filename_split[1])
    key.append(filename_split[2])
    collection_address = stac_challenge_folder +'/' + filename
    geometry_item = {
           "type": "Polygon",
           "coordinates": bbox
        
         }
    bbox_item = bbox
    return id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item

In [31]:
# create a Colllection object with JSON
def create_corresponding_item(id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item):
    item_json = {
        "id": id_item,
        "type": type_item,
        "geometry": geometry_item,
        "bbox": bbox_item,
        "properties": properties_item,
        "links": links_item,
        "assets":assets_item,
    }


    it= Item(item_json)
    return it

### Test

In [32]:
collection = Collection.open(collection_address)
bbox = get_bbox(stac_challenge_folder + '/'+ 'accra_1', "accra_1_drains.geojson")
id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item = get_tags_items(collection,"accra_1", "accra_1_drains.geojson", bbox)
it = create_corresponding_item(id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item)

In [33]:
item_address = stac_challenge_folder +'/' + "accra_1_drains" + '-item.json'
it.save_as(item_address)
print(it.id)
print(it.filename)
print(it.path)
item = Item.open(item_address)

accra_1_drains
/Users/axel/Desktop/STAC_Challenge_2/accra_1_drains-item.json
/Users/axel/Desktop/STAC_Challenge_2


In [34]:
item.assets

{'raster': {'title': 'image',
  'href': 'https://oin-hotosm.s3.amazonaws.com/5b694a0f4b87366cc0f0fa70/0/5b694a0f4b8736ebfff0fa71.tif',
  'type': 'image/vnd.stac.geotiff; cloud-optimized=true'},
 'vector': {'title': 'accra_1_drains',
  'href': '/Users/axel/Desktop/STAC_Challenge_2/accra_1/accra_1_drains.geojson',
  'type': 'application/geo+json'}}

# We go through the files to create all the items and collections


In [43]:


for folder in os.listdir(stac_challenge_folder):
    if(folder!= '.DS_Store' and "json" not in folder):
        print(folder)
        
        stac_version, id_collection, title, description, keywords, version, license, providers, extent, links = get_tags_collection(folder)
        collection = create_corresponding_collection(stac_version, id_collection, title, description, keywords, version, license, providers, extent, links)
        #collection_address = stac_challenge_folder +'/' + folder + '_collection.json'
        collection_address = stac_challenge_folder +'/' +id_collection_basic+folder+'/'+ folder + '_collection.json'
        collection.save_as(collection_address)
        catalog.add_catalog(collection)
        
        for file in os.listdir(stac_challenge_folder+'/'+folder):
            print(file)
            
            bbox = get_bbox(stac_challenge_folder + '/'+ folder, file)
            id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item = get_tags_items(collection,folder, file, bbox)
            item = create_corresponding_item(id_item, type_item, geometry_item, bbox_item, properties_item, links_item,assets_item)
            item_address = stac_challenge_folder +'/' +id_collection_basic+folder+'/'+ file.split('.')[0] + '-item.json'
            item.save_as(item_address)
            collection.add_item(item)
            
        #collection_address = stac_challenge_folder +'/' + folder + '_collection.json'
        collection.save_as(collection_address)
catalog.save_as(catalog_address)


accra_1
challenge_collection_for_accra_1
accra_1_buildings.geojson
accra_1_drains.geojson


challenge_catalog

'/Users/axel/Desktop/STAC_Challenge_2/catalog.json'

In [36]:
item.data['links']

[{'rel': 'self',
  'href': '/Users/axel/Desktop/STAC_Challenge_2/challenge_collection_for_accra_1/accra_1_drains.json'},
 {'rel': 'root', 'href': '../catalog.json'},
 {'rel': 'parent', 'href': 'catalog.json'},
 {'rel': 'collection', 'href': 'catalog.json'}]